In [1]:
!pip install evaluate seqeval -q

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
import evaluate
import numpy as np

seqeval = evaluate.load("seqeval")

2024-04-09 02:24:23.237189: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 02:24:23.237302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 02:24:23.363470: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Dataset

In [3]:
wnut = load_dataset("wnut_17")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

Dataset wnut_17 downloaded and prepared to /root/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# a dict of Dataset
wnut

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [5]:
wnut['train'].select(range(5)).to_pandas()

,id,tokens,ner_tags
0,0,"[@paulwalk, It, 's, the, view, from, where, I,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, ..."
1,1,"[From, Green, Newsfeed, :, AHFA, extends, dead...","[0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,2,"[Pxleyes, Top, 50, Photography, Contest, Pictu...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,3,"[today, is, my, last, day, at, the, office, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,4,"[4Dbling, 's, place, til, monday, ,, party, pa...","[9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [6]:
wnut['test'].select(range(5)).to_pandas()

,id,tokens,ner_tags
0,0,"[&, gt, ;, *, The, soldier, was, killed, when,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,"[&, gt, ;, *, Police, last, week, evacuated, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, ..."
2,2,"[&, gt, ;, *, The, army, on, Thursday, recover...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,"[&, gt, ;, *, The, four, civilians, killed, in...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[The, bodies, of, the, soldiers, were, recover...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, ..."


In [7]:
label_list = wnut['train'].features['ner_tags'].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

# Tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
example = wnut['train'][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)

In [10]:
tokenized_input

{'input_ids': [101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
print(example['tokens'])
print("========================================")
print(example['ner_tags'])
print("========================================")
print(tokenizer.convert_ids_to_tokens(tokenized_input['input_ids']))

['@paulwalk', 'It', "'s", 'the', 'view', 'from', 'where', 'I', "'m", 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'ESB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]
['[CLS]', '@', 'paul', '##walk', 'it', "'", 's', 'the', 'view', 'from', 'where', 'i', "'", 'm', 'living', 'for', 'two', 'weeks', '.', 'empire', 'state', 'building', '=', 'es', '##b', '.', 'pretty', 'bad', 'storm', 'here', 'last', 'evening', '.', '[SEP]']


In [12]:
# get token labels
# for a word that split into multiple tokens, only the first one is labeled, and the subsequent will
# get -100
def tokenize_and_align_labels(examples):
    # each input row is a dictionary
    tokenized_input = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_idx = tokenized_input.word_ids(batch_index=i)
#         print("got label", label)
        prev_wid = None
        label_id = []
        for wid in word_idx:
            if wid is None:
                label_id.append(-100)
            elif wid != prev_wid:
                label_id.append(label[wid])
            else:
                label_id.append(-100)
            prev_wid = wid
        labels.append(label_id)
    tokenized_input['labels'] = labels
    return tokenized_input

In [13]:
tokenize_and_align_labels(wnut['train'][0:2])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], [101, 2013, 2665, 2739, 7959, 2098, 1024, 6289, 7011, 8908, 15117, 2005, 10878, 2400, 2000, 13292, 1012, 1019, 8299, 1024, 1013, 1013, 4714, 3126, 2140, 1012, 4012, 1013, 2484, 8490, 3501, 22025, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, -100, -100, 0, 0, -100, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, -100, 0, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, -100, -100, 0, 5, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]]}

In [14]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

## Collator

In [15]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluate

In [16]:
label_list = wnut["train"].features[f"ner_tags"].feature.names

labels = [label_list[i] for i in example['ner_tags']]
print(labels)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'O', 'B-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [17]:
def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=2)
    
    pred_labels = [
        [label_list[p] for p, l in zip(pred, label) if l != -100]
        for pred, label in zip(preds, labels)
    ]
    
    actual_labels = [
        [label_list[l] for p, l in zip(pred, label) if l != -100]
        for pred, label in zip(preds, labels)
    ]
    
    results = seqeval.compute(predictions=pred_labels, references=actual_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Train

In [18]:
id2label = {i:label for i, label in enumerate(label_list)}
label2id = {label:i for i, label in enumerate(label_list)}
print(id2label)
print(label2id)

{0: 'O', 1: 'B-corporation', 2: 'I-corporation', 3: 'B-creative-work', 4: 'I-creative-work', 5: 'B-group', 6: 'I-group', 7: 'B-location', 8: 'I-location', 9: 'B-person', 10: 'I-person', 11: 'B-product', 12: 'I-product'}
{'O': 0, 'B-corporation': 1, 'I-corporation': 2, 'B-creative-work': 3, 'I-creative-work': 4, 'B-group': 5, 'I-group': 6, 'B-location': 7, 'I-location': 8, 'B-person': 9, 'I-person': 10, 'B-product': 11, 'I-product': 12}


In [19]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", 
    num_labels=len(label_list), 
    id2label=id2label, 
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
training_args = TrainingArguments(
    output_dir="distilbert_wnut_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=100,
#     save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none" # disable wandb.ai login
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,No log,0.314364,0.213483,0.017609,0.032534,0.928092
200,No log,0.291632,0.595556,0.248378,0.350556,0.939122
300,No log,0.275789,0.627376,0.305839,0.411215,0.942114
400,No log,0.268236,0.614414,0.316033,0.417381,0.942328


/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=426, training_loss=0.20434097504951584, metrics={'train_runtime': 37.4192, 'train_samples_per_second': 181.404, 'train_steps_per_second': 11.385, 'total_flos': 91781128898820.0, 'train_loss': 0.20434097504951584, 'epoch': 2.0})

In [22]:
trainer.save_model('distilbert')

## Inference

In [23]:
model_path = "/kaggle/working/distilbert"

In [21]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

In [45]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
inputs = tokenizer(text, return_tensors='pt')
inputs

{'input_ids': tensor([[ 101, 1996, 3585, 2110, 6424, 2024, 2019, 2137, 2658, 3455, 2136, 2241,
         1999, 2624, 3799, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [33]:
import torch

model = AutoModelForTokenClassification.from_pretrained(model_path)
with torch.no_grad():
    logits = model(**inputs).logits

# batch_size x N_token x N_class
logits.shape

torch.Size([1, 17, 13])

In [55]:
# batch_size x N_token
preds = torch.argmax(logits, axis=2)
labels = [id2label[p.item()] for p in preds[0]]
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].tolist()[0])

print({t:l for t, l in zip(tokens, labels)})

{'[CLS]': 'B-group', 'the': 'B-group', 'golden': 'B-location', 'state': 'B-location', 'warriors': 'B-group', 'are': 'O', 'an': 'O', 'american': 'B-location', 'professional': 'O', 'basketball': 'O', 'team': 'O', 'based': 'O', 'in': 'O', 'san': 'B-location', 'francisco': 'B-location', '.': 'O', '[SEP]': 'O'}


In [56]:
# batch inference


In [62]:
ds_test = wnut['test']
tokenizer = AutoTokenizer.from_pretrained(model_path)


def tokenize(row):
    tokenized_inputs = tokenizer(row['tokens'], truncation=True, is_split_into_words=True)
    return tokenized_inputs
    
    
ds_test = ds_test.map(tokenize, num_proc=2)

#0:   0%|          | 0/644 [00:00<?, ?ex/s]

#1:   0%|          | 0/643 [00:00<?, ?ex/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [66]:
model = AutoModelForTokenClassification.from_pretrained(model_path)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=1, 
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [70]:
preds = trainer.predict(ds_test)
preds = preds.predictions
preds.shape

(1287, 191, 13)

In [71]:
preds = np.argmax(preds, axis=2)
preds.shape

(1287, 191)

In [74]:
labels = [id2label[p] for p in preds[0]]
tokens = tokenizer.convert_ids_to_tokens(ds_test[0]['input_ids'])

In [77]:
l = min(len(tokens), len(labels))
print({t:l for t, l in zip(tokens[:l], labels[:l])})

{'[CLS]': 'O', '&': 'O', 'gt': 'O', ';': 'O', '*': 'O', 'the': 'O', 'soldier': 'O', 'was': 'O', 'killed': 'O', 'when': 'O', 'another': 'O', 'avalanche': 'O', 'hit': 'O', 'an': 'O', 'army': 'O', 'barracks': 'O', 'in': 'O', 'northern': 'O', 'area': 'O', 'of': 'O', 'son': 'B-location', '##mar': 'B-location', '##g': 'I-location', ',': 'O', 'said': 'O', 'a': 'O', 'military': 'O', 'spokesman': 'O', '.': 'O', '[SEP]': 'O'}
